In [41]:
import KNNclassy
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
from sklearn.neighbors import NearestNeighbors
importlib.reload(KNNclassy)

<module 'KNNclassy' from '/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/KNNclassy.py'>

In [42]:
frac_testing = 0.3
nneigh = [10]
pathClassy = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/"
pathData = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/input/"
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]
EOSgrey = ["APR4_EPP",  "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLY2", "SLY9", "SLY230A"]
EOScolor = ["BHF_BBB2", "MS1_PP", "SLy"]
color_v = ['orange', 'green', 'blue']
BayesFactor = [1.526, 1.555, 0.056, 1.422, 1.177, 1.283, 0.276, 0.001, 0.009, 0.176, 0.179, 0.159, 0.108, 0.107, 0.33, 0.025, 0.288, 0.29, 0.618, 1.0, 1.028, 0.37, 0.932]

In [43]:
EOSdic = {}
count = 0
#EOS = ['SLy']

In [44]:
for eos in EOS:
    
    print("Doing", eos)
    print('*'*60)

    for kk in nneigh:
        
        print("Nº of neighbors: ", kk)
        print('-'*40)

        KNN = KNNclassy.ClassificationKNN()
        KNN.load_original_dataset(pathData,eos+"/EMB/original_data_"+eos+"_s300_f0d7.csv",frac_testing)
    
        KNN.loadModel(pathClassy+"optimal_models_final/", "knn_3cat_grid_eos_"+eos)
        new_dic = {}
        new_dic['knn'] = KNN
        new_dic['bayes'] = BayesFactor[count]
        EOSdic[eos] = new_dic
    
        print('*'*60)
        
    count = count + 1
        

Doing APR4_EPP
************************************************************
Nº of neighbors:  10
----------------------------------------
File shape :  (198961, 11)
[ 1.006907  1.726921  1.172679 ... 79.196968 66.51236  59.432453]
************************************************************
Loading data...
Nº of features:  5
Nº of events:  198961
Nº of events for training:  139273
Nº of events for testing:  59688
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final/knn_3cat_grid_eos_APR4_EPP.joblib
************************************************************
Doing BHF_BBB2
************************************************************
Nº of neighbors:  10
----------------------------------------
File shape :  (193624, 11)
[ 1.006907  1.726921  1.172679 ... 79.196968 66.51236  59.432453]
************************************************************
Loading data...
Nº of features:  

File shape :  (201373, 11)
[ 1.006907  1.726921  1.172679 ... 79.196968 66.51236  59.432453]
************************************************************
Loading data...
Nº of features:  5
Nº of events:  201373
Nº of events for training:  140962
Nº of events for testing:  60411
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final/knn_3cat_grid_eos_SKI3.joblib
************************************************************
Doing SKI4
************************************************************
Nº of neighbors:  10
----------------------------------------
File shape :  (199255, 11)
[ 1.006907  1.726921  1.172679 ... 79.196968 66.51236  59.432453]
************************************************************
Loading data...
Nº of features:  5
Nº of events:  199255
Nº of events for training:  139479
Nº of events for testing:  59776
Datasets loaded!
*************************************

In [88]:
def matrix(algo,nneigh,eos):
    total_events = len(algo.label_test)
    nplus_NO = np.zeros(total_events)
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    
    for i in range(0,total_events):
        distance, closestn = algo.model.kneighbors(algo.xtest[i].reshape(1, -1),n_neighbors=nneigh,return_distance = True)
     #   print(1/distance[0])
        if (distance.any() <= 0.001):
            print('event '+str(i)+' gives problems')
            print(distance[0])
        weighted_neighs = 1/distance[0]
        norm = np.sum(1/distance[0])
        labels = algo.label_train[closestn][0].tolist()
        for ll in range(0,len(labels)): 
            labels[ll] = int(labels[ll])
        labels = np.array(labels)
        ind_0 = np.where(labels == 0)
        ind_1 = np.where(labels == 1)
        ind_2 = np.where(labels == 2)
    
        neighs_0 = weighted_neighs[ind_0]
        neighs_1 = weighted_neighs[ind_1]
        neighs_2 = weighted_neighs[ind_2]

        
        nplus_NO[i] = np.sum(neighs_0)/norm
        nplus_NS[i] = np.sum(neighs_1)/norm
        nplus_REM[i] = np.sum(neighs_2)/norm
        
    
    with open('matrix_events_EOS-'+eos+'.txt','w+') as f:

        f.write('# HasNS_true \t HasREM_true \t f_0 \t f_1 \t f_2 \n')

        for i in range(0,total_events):

            f.write('%d \t %d \t %.4f \t %.4f \t %.4f \n'%(algo.label_NStrue[i],algo.label_REMtrue[i],nplus_NO[i],nplus_NS[i], nplus_REM[i]))
    
    return 

In [ ]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    matrix(EOSdic[eos]['knn'],nneigh[0],eos)
    print('#'*60)

Doing EOS:  APR4_EPP


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:28: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  BHF_BBB2


/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:13: RuntimeWarning: divide by zero encountered in true_divide
  weighted_neighs = 1/distance[0]
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:14: RuntimeWarning: divide by zero encountered in true_divide
  norm = np.sum(1/distance[0])
/var/folders/jl/s9jkbh4j24xc_01t8c28q52m0000gn/T/ipykernel_90995/587142062.py:28: RuntimeWarning: invalid value encountered in double_scalars
  nplus_NO[i] = np.sum(neighs_0)/norm


############################################################
Doing EOS:  H4
